In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nothing').getOrCreate()
D1=[(1,1,"N"),
    (2,1,"Y"),
    (2,2,"N"),
    (3,3,"N"),
    (4,2,"N"),
    (4,3,"Y"),
    (4,4,"N")
    ]
columns=["employee_id","department_id","primary_flag"]
df=spark.createDataFrame(data=D1,schema=columns)
df.show()

+-----------+-------------+------------+
|employee_id|department_id|primary_flag|
+-----------+-------------+------------+
|          1|            1|           N|
|          2|            1|           Y|
|          2|            2|           N|
|          3|            3|           N|
|          4|            2|           N|
|          4|            3|           Y|
|          4|            4|           N|
+-----------+-------------+------------+



In [6]:
df1=df.groupBy("employee_id").agg(count("employee_id").alias("dept_count"))
df1.show()

+-----------+----------+
|employee_id|dept_count|
+-----------+----------+
|          1|         1|
|          2|         2|
|          3|         1|
|          4|         3|
+-----------+----------+



In [8]:
join_df=df1.join(df,df1.employee_id==df.employee_id).drop(df.employee_id)
join_df.show()

+----------+-----------+-------------+------------+
|dept_count|employee_id|department_id|primary_flag|
+----------+-----------+-------------+------------+
|         1|          1|            1|           N|
|         2|          2|            1|           Y|
|         2|          2|            2|           N|
|         1|          3|            3|           N|
|         3|          4|            2|           N|
|         3|          4|            3|           Y|
|         3|          4|            4|           N|
+----------+-----------+-------------+------------+



In [11]:
final_df=join_df.filter((F.col("dept_count")==1) |(F.col("primary_flag")=='Y')).drop("dept_count","primary_flag")
final_df.show()

+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          1|            1|
|          2|            1|
|          3|            3|
|          4|            3|
+-----------+-------------+



In [32]:
data=[("kasireddy naidu",),
 ("konidela ram charan",),
  ("Nandamuri tarak ramarao",),
   ("charan",)]
columns=["customer_name"]

df=spark.createDataFrame(data,columns)
df.show()

df.withColumn("first_name",when(size(split("customer_name"," "))>=2,split(col("customer_name")," ")[0]))\
  .withColumn("middle_name",when(size(split("customer_name"," "))==3,split(col("customer_name")," ")[1]))\
  .withColumn("last_name",
              coalesce(
                  when(size(split("customer_name"," "))==3,split("customer_name"," ")[2]),
                  when(size(split("customer_name"," "))==2,split("customer_name"," ")[1]),
                  when(size(split("customer_name"," "))==1,split("customer_name"," ")[0])
                 )
              ).show()

+--------------------+
|       customer_name|
+--------------------+
|     kasireddy naidu|
| konidela ram charan|
|Nandamuri tarak r...|
|              charan|
+--------------------+

+--------------------+----------+-----------+---------+
|       customer_name|first_name|middle_name|last_name|
+--------------------+----------+-----------+---------+
|     kasireddy naidu| kasireddy|       NULL|    naidu|
| konidela ram charan|  konidela|        ram|   charan|
|Nandamuri tarak r...| Nandamuri|      tarak|  ramarao|
|              charan|      NULL|       NULL|   charan|
+--------------------+----------+-----------+---------+



In [34]:
data=[(1,101,500.0,"2024-01-01"),
      (1,101,600.0,"2024-01-01"),
      (2,102,200.0,"2024-01-02"),
      (3,101,300.0,"2024-01-03"),
      (4,103,100.0,"2024-01-04"),
      (5,102,400.0,"2024-01-05"),
      (6,103,600.0,"2024-01-06"),
      (7,101,200.0,"2024-01-07")

]

columns=['t_id','user_id','amount','t_date']
df=spark.createDataFrame(data,columns)
df.show()

+----+-------+------+----------+
|t_id|user_id|amount|    t_date|
+----+-------+------+----------+
|   1|    101| 500.0|2024-01-01|
|   1|    101| 600.0|2024-01-01|
|   2|    102| 200.0|2024-01-02|
|   3|    101| 300.0|2024-01-03|
|   4|    103| 100.0|2024-01-04|
|   5|    102| 400.0|2024-01-05|
|   6|    103| 600.0|2024-01-06|
|   7|    101| 200.0|2024-01-07|
+----+-------+------+----------+



In [35]:
df1=df.select("user_id","t_date").distinct()
df1.show()

+-------+----------+
|user_id|    t_date|
+-------+----------+
|    101|2024-01-01|
|    101|2024-01-03|
|    102|2024-01-02|
|    103|2024-01-06|
|    103|2024-01-04|
|    101|2024-01-07|
|    102|2024-01-05|
+-------+----------+



In [37]:
df2=df1.groupBy("user_id").agg(count("t_date").alias("trans_count"))\
                          .filter(col("trans_count")>=3)
df2.show()

+-------+-----------+
|user_id|trans_count|
+-------+-----------+
|    101|          3|
+-------+-----------+



In [40]:
join_df=df2.join(df,df2.user_id==df.user_id)\
           .drop(df2.user_id,df2.trans_count,"t-id","t_date")\
           .groupBy("user_id")\
           .agg(avg("amount").alias("total_spend"))
join_df.show()3

+-------+-----------+
|user_id|total_spend|
+-------+-----------+
|    101|      400.0|
+-------+-----------+



In [44]:
data=[('RCB',),('CSK',),('MI',),('PBKS',)]
df=spark.createDataFrame(data,['team'])
df.show()

df1=df.alias("t1").crossJoin(df.alias("t2"))\
      .where(F.col("t1.team")!=F.col("t2.team"))\
      .select(F.col("t1.team").alias("team1"),F.col("t2.team").alias("team2"))
df1.show()
df2=df1.filter(F.col("team1")>F.col("team2"))
df2.show()

+----+
|team|
+----+
| RCB|
| CSK|
|  MI|
|PBKS|
+----+

+-----+-----+
|team1|team2|
+-----+-----+
|  RCB|  CSK|
|  CSK|  RCB|
|  RCB|   MI|
|  RCB| PBKS|
|  CSK|   MI|
|  CSK| PBKS|
|   MI|  RCB|
|   MI|  CSK|
| PBKS|  RCB|
| PBKS|  CSK|
|   MI| PBKS|
| PBKS|   MI|
+-----+-----+

+-----+-----+
|team1|team2|
+-----+-----+
|  RCB|  CSK|
|  RCB|   MI|
|  RCB| PBKS|
|   MI|  CSK|
| PBKS|  CSK|
| PBKS|   MI|
+-----+-----+

